In [112]:
from __future__ import division
import pandas as pd
import math
from gensim.models import coherencemodel
from gensim.models import Word2Vec, KeyedVectors
import gc
import pickle
from scipy.linalg import norm
from sklearn.preprocessing import normalize
from scipy.stats import entropy
from sklearn.metrics.pairwise import cosine_similarity
import os
from itertools import combinations
import numpy as np
from gensim import corpora, models, similarities #calc all similarities at once, from http://radimrehurek.com/gensim/tut3.html
from sklearn.metrics.pairwise import cosine_similarity
from random import seed, sample
import seaborn as sns
from ksvd import ApproximateKSVD 
import random
import ast
import html
import re
from sklearn.decomposition import TruncatedSVD
import ast
from tqdm import tqdm
#np.set_printoptions(threshold=np.inf) #set to print full output
import numpy as np
random.seed(42)
#os.chdir('/home/ll16598/Documents/Altered_States_Reddit/model_pipeline')
import sys
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModel  # Import AutoTokenizer and AutoModel
import nltk
from collections import Counter
from scipy.stats import zscore
#nltk.download('stopwords')
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
from itertools import combinations
import networkx as nx
import umap
from mpl_toolkits.mplot3d import Axes3D
from nltk.corpus import stopwords
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as stats
import gc  # Garbage collection module
from scipy.stats import sem
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

#sys.path.append('/home/ll16598/Documents/Altered_States_Reddit/model_pipeline/__pycache__')
#from quality import reconst_qual, topic_diversity, coherence_centroid, coherence_pairwise #written for this jupyter notebook

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ll16598/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [113]:
import gudhi as gd
import gudhi.representations

def compute_persistence_diagram(data):
    # Compute the Rips complex
    rips_complex = gd.RipsComplex(points=data, max_edge_length=2.0)
    # Construct a simplex tree
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)
    # Compute persistent homology
    persistence = simplex_tree.persistence()
    # Plot persistence diagram`
    gd.plot_persistence_diagram(persistence)
    gd.plot_persistence_barcode(persistence)

    plt.show()
    return persistence

def add_geometric_centroid(data):
    """
    Given an (N, D) array 'data' of N points in D dimensions,
    computes the mean (geometric centroid) and appends it
    as an extra row at the end.

    Returns:
      data_with_centroid: an (N+1, D) array,
        where the last row is the centroid.
      centroid_index: the integer index of the new centroid row.
    """
    centroid = np.mean(data, axis=0)             # shape (D,)
    data_with_centroid = np.vstack([data, centroid])
    centroid_index = data_with_centroid.shape[0] - 1
    return data_with_centroid, centroid_index

def build_centroid_distance_matrix(data_with_centroid, centroid_index, large_val=1e6):
    """
    Creates a distance matrix where only edges from the 'centroid_index'
    to other points have the real Euclidean distance.
    All other pairwise distances are set to 'large_val'.
    """
    N = data_with_centroid.shape[0]
    dist_matrix = np.full((N, N), large_val, dtype=float)

    # Diagonal = 0
    np.fill_diagonal(dist_matrix, 0.0)

    # Compute distances between centroid and each other point
    for i in range(N):
        if i == centroid_index:
            continue
        # Real distance from centroid -> i
        dist = cosine_similarity(data_with_centroid[centroid_index].reshape(1,-1),\
                                 data_with_centroid[i].reshape(1,-1))
        dist_matrix[centroid_index, i] = dist
        dist_matrix[i, centroid_index] = dist

    return dist_matrix

def compute_persistence_centroid(data, max_edge_length=2.0, plotting=True):
    """
    1) Compute the geometric centroid of 'data' and append it as an extra row.
    2) Build a distance matrix such that only the centroid can connect to other points.
    3) Construct the Rips complex using the custom distance matrix.
    4) Compute and plot the persistence diagram and barcode.

    Returns:
      persistence: The list of (dim, (birth, death)) intervals from GUDHI
    """
    # 1) Add the centroid
    data_with_centroid, centroid_index = add_geometric_centroid(data)

    # 2) Build the custom distance matrix
    dist_matrix = build_centroid_distance_matrix(data_with_centroid, centroid_index, large_val=1e6)

    # 3) Create RipsComplex from the distance matrix
    rips_complex = gd.RipsComplex(distance_matrix=dist_matrix, max_edge_length=max_edge_length)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
    del dist_matrix  # Delete objects
    gc.collect()  # Force garbage collection to free memory
    # 4) Compute persistent homology
    persistence = simplex_tree.persistence()
    if not plotting:
        return rips_complex, simplex_tree, persistence
    # 5) Plot the persistence diagram & barcode
    gd.plot_persistence_diagram(persistence)
    gd.plot_persistence_barcode(persistence)
    plt.show()

    return rips_complex, simplex_tree, persistence


def get_alive_components_over_scales(births, deaths, step=0.025):
    """
    Given lists of (birth, death) intervals for a particular homology dimension,
    compute how many such features (e.g., connected components if D=0, loops if D=1, etc.)
    are 'alive' at increments of 'step' from 0 up to max(deaths).

    Returns:
    - scales: list of scale values (0, 0.025, 0.05, ...)
    - alive_counts: corresponding list of how many features are alive at each scale
    """
    if len(births) == 0:
        # No intervals => no features
        return [], []
    
    max_death = max(deaths)
    scales = np.arange(0, max_death + 1e-9, step)
    
    alive_counts = []
    for s in scales:
        # Count intervals that are alive: birth <= s < death
        count_alive = sum(1 for (b, d) in zip(births, deaths) if b <= s < d)
        alive_counts.append(count_alive)
    
    return list(scales), alive_counts

def get_rips_time(df, embeddings='sentence_embeddings_pca', step=0.05, D=0):
    """
    For each row in df, build a Rips complex, extract dimension-D intervals
    (e.g., D=0 => connected components, D=1 => loops, etc.),
    then compute how many such features are 'alive' at increments of 'step'.

    Creates two new columns in df:
    - f"scales_dim{D}": The scale values
    - f"alive_dim{D}": The counts of alive features at each scale
    """
    # Copy the DataFrame to avoid SettingWithCopy warnings
    df = df.copy()
    
    # Prepare two new columns (lists)
    df[f'scales_dim{D}'] = None
    df[f'alive_dim{D}'] = None
    df['rt'] = None

    for idx, row in df.iterrows():
        # Get the embeddings for this row
        embed = row[embeddings]
        if not isinstance(embed, (list, np.ndarray)) or len(embed) == 0:
            continue
        
        # Build the Rips Complex
        rips_complex = gd.RipsComplex(points=embed, max_edge_length=5.0)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)

        # Extract dimension-D intervals from persistence
        persistence = simplex_tree.persistence()
        
        births_dimD = []
        deaths_dimD = []
        
        for dim, (b, d) in persistence:
            if dim == D and d != float('inf'):  # ignoring infinite intervals
                births_dimD.append(b)
                deaths_dimD.append(d)

        # Compute how many features are alive at each scale
        scales, alive_components = get_alive_components_over_scales(births_dimD, deaths_dimD, step=step)
        if len(deaths_dimD)>0:
            df.at[idx, f"rt"] = max(deaths_dimD)

        # Store these lists in the new columns
        df.at[idx, f"scales_dim{D}"] = scales
        df.at[idx, f"alive_dim{D}"] = alive_components

    return df
def get_rips_time_centroid(df, embeddings='sentence_embeddings_pca', step=0.05, D=0):
    """
    For each row in df, build a Rips complex, extract dimension-D intervals
    (e.g., D=0 => connected components, D=1 => loops, etc.),
    then compute how many such features are 'alive' at increments of 'step'.

    Creates two new columns in df:
    - f"scales_dim{D}": The scale values
    - f"alive_dim{D}": The counts of alive features at each scale
    """
    # Copy the DataFrame to avoid SettingWithCopy warnings
    df = df.copy()
    
    # Prepare two new columns (lists)
    df[f'centroid_scales_dim{D}'] = None
    df[f'centroid_alive_dim{D}'] = None

    for idx, row in df.iterrows():
#         if idx in [26, 27, 28]:
#             print('warning: skipping a computationally intensive sample')
#             continue
        # Get the embeddings for this row
        embed = row[embeddings]
        if not isinstance(embed, (list, np.ndarray)) or len(embed) == 0:
            continue
        
        # Build the Rips Complex
        rips_complex, simplex_tree,persistence= compute_persistence_centroid(embed,max_edge_length=4,plotting=False)
        births_dimD = []
        deaths_dimD = []
        
        for dim, (b, d) in persistence:
            if dim == D and d != float('inf'):  # ignoring infinite intervals
                births_dimD.append(b)
                deaths_dimD.append(d)

        # Compute how many features are alive at each scale
        scales, alive_components = get_alive_components_over_scales(births_dimD, deaths_dimD, step=step)
        
        # Store these lists in the new columns
        df.at[idx, f"centroid_scales_dim{D}"] = scales
        df.at[idx, f"centroid_alive_dim{D}"] = alive_components
        del rips_complex, simplex_tree, persistence, births_dimD, deaths_dimD, scales, alive_components  # Delete objects
        gc.collect()  # Force garbage collection to free memory
        
        #time.sleep(0.2)  # Pause for 0.5 seconds
    return df

import hypernetx as hnx  # Hypergraph library
import itertools
def get_rips_complex_G(df, embedding=str('sentence_embeddings')):
    df['graph']=None
    df['density']=None
    df['simplex_tree']=None
    df['hypergraph'] = None
    for idx, row in df.iterrows():
        G=nx.Graph()
        H = {}  # Hypergraph as a dictionary: {hyperedge_id: [vertices]}
        embed = row[embedding] 
        edge_length=row['rt']
        if not edge_length:
              continue
        rips_complex = gd.RipsComplex(points=embed, max_edge_length=edge_length)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)
        hyperedges = set()
        
        for simplex, fvalue in simplex_tree.get_filtration():  # Get all simplices
            if len(simplex) > 1:  # Ignore single nodes
                hyperedges.add(frozenset(simplex))  # Store as a hyperedge
        
        # Convert hyperedges to a hypergraph representation
        H = {f'he_{i}': list(hyperedge) for i, hyperedge in enumerate(hyperedges)}
        hypergraph = hnx.Hypergraph(H)


        # Store in DataFrame
        df.at[idx, 'hypergraph'] = hypergraph
        #df.at[idx, 'hyper_density'] = hypergraph.edge_size()
        
        
        edges = []
        
        for simplex, fvalue in simplex_tree.get_skeleton(3):
            if len(simplex) >= 2:
                # Create every pair of vertices in this simplex
                for (i, j) in itertools.combinations(simplex, 2):
                    # You can choose which filtration value to assign as weight
                    # Here, we use the simplex's overall fvalue.
                    # Alternatively, you might want the actual distance between i,j if known.
                    G.add_edge(i, j, weight=fvalue)
                    edges.append((i, j, fvalue))
        df['graph'].loc[idx]=G
        df['density'].loc[idx]=nx.density(G)
        df['simplex_tree'].loc[idx]=simplex_tree

    return df#df

def compute_euler_characteristic(simplex_tree, max_dim=3):
    """
    Compute the Euler characteristic of a simplicial complex represented by a GUDHI simplex tree.
    
    Parameters:
      simplex_tree: A GUDHI simplex tree containing simplices up to dimension max_dim.
      max_dim: Maximum dimension to consider (e.g., 3 for tetrahedra).
    
    Returns:
      euler: The Euler characteristic computed as 
             f0 - f1 + f2 - f3 + ... (up to max_dim).
    """
    if not simplex_tree:
        return None
    # Dictionary to store counts for each dimension
    simplex_counts = {}
    for d in range(max_dim + 1):
        # In GUDHI, a d-simplex is a simplex with d+1 vertices.
        simplices_d = [simplex for simplex, filt in simplex_tree.get_skeleton(d) if len(simplex) == d + 1]
        simplex_counts[d] = len(simplices_d)
    #    print(f"Number of {d}-simplices (f_{d}): {simplex_counts[d]}")
    
    # Euler characteristic: sum_{d=0}^{max_dim} (-1)^d * f_d
    euler = sum(((-1) ** d) * simplex_counts[d] for d in range(max_dim + 1))
    return euler

from scipy.stats import pearsonr, spearmanr
def get_node_join_fvalues(G):
    """
    Extracts the filtration value (fvalue) at which each node joins the next node
    in the sorted order of nodes.

    Parameters:
    - G: networkx.Graph (a graph with weighted edges where weights are fvalues)

    Returns:
    - fvalue_list: List of tuples (node_i, node_j, fvalue) representing
      the filtration value at which node_i joins node_j.
    """
    if not G:
        return None, None
    sorted_nodes = sorted(G.nodes())  # Sort nodes in order
    fvalue_list = []

    for i in range(len(sorted_nodes) - 1):
        n1, n2 = sorted_nodes[i], sorted_nodes[i + 1]  # Get consecutive nodes

        if G.has_edge(n1, n2):  # Check if edge exists
            fvalue = G[n1][n2]['weight']  # Get filtration value (edge weight)
            fvalue_list.append(fvalue)

    return sorted_nodes,fvalue_list

def compute_correlations(sorted_nodes, fvalues):
    """
    Computes Pearson and Spearman correlation coefficients between node indices and f-values.

    Parameters:
    - sorted_nodes: List of node indices (sorted order)
    - fvalues: List of f-values corresponding to edges between consecutive nodes

    Returns:
    - pearson_corr: Pearson correlation coefficient
    - spearman_corr: Spearman correlation coefficient
    """
    if not fvalues:
        return np.nan, np.nan
    if len(fvalues) < 2:  # Correlations need at least two data points
        return np.nan, np.nan

    indices = np.arange(len(fvalues))  # Create index positions (0,1,2,...) for correlation

    pearson_corr, _ = pearsonr(indices, fvalues)
    spearman_corr, _ = spearmanr(indices, fvalues)

    return pearson_corr, spearman_corr

import networkx as nx
import numpy as np
import community  # for Louvain modularity detection (python-louvain)
from networkx.algorithms import approximation
from scipy.stats import pearsonr, spearmanr

import collections
import itertools
def ff3(x):
    return x*(x-1)*(x-2)

def avg_tetr_cc(g):
    tetrahedra = itertools.islice(itertools.groupby(
        nx.enumerate_all_cliques(g), len), 3, 4)
    try:
        tetrahedra = next(tetrahedra)[1]
    except StopIteration:
        return 0
    cnts = collections.Counter(itertools.chain(*tetrahedra))
    return 6 * sum(cnt / ff3(g.degree[v]) for v, cnt in cnts.items()) / len(g)



from scipy.linalg import eigvalsh

def compute_graph_metrics(G):
    """
    Computes various network metrics for a given graph G, including Laplacian eigenvalues.
    
    Metrics:
    - Shortest Path (Weighted & Unweighted)
    - Number of Triangles
    - Number of Tetrahedra (4-cliques)
    - Modularity using Louvain (Weighted)
    - Clustering Coefficient
    - Max & Mean Degree
    - Max & Mean Betweenness Centrality
    - Max & Mean Strength (Weighted Degree)
    - Second Smallest Laplacian Eigenvalue (Fiedler Value)
    - Largest Laplacian Eigenvalue

    Parameters:
    - G (networkx.Graph): A 3-skeleton graph with weighted edges.

    Returns:
    - Dictionary with computed graph metrics.
    """
    metrics = {
        "shortest_path_unweighted": np.nan,
        "shortest_path_weighted": np.nan,
        "num_triangles": np.nan,
        "num_tetrahedra": np.nan, #maybe also area of these
        "modularity_louvain": np.nan,
        "clustering_coefficient": np.nan,
        "max_degree": np.nan,
        "mean_degree": np.nan,
        "max_betweenness": np.nan,
        "mean_betweenness": np.nan,
        "max_strength": np.nan,
        "mean_strength": np.nan,
        "fiedler_value": np.nan,
        "largest_laplacian_eigenvalue": np.nan
    }

    if not G or G.number_of_nodes() < 2:
        return metrics

    # Sorted nodes
    sorted_nodes = sorted(G.nodes())

    # Shortest Path (Unweighted & Weighted)
    first_node, last_node = sorted_nodes[0], sorted_nodes[-1]
    if nx.has_path(G, first_node, last_node):
        metrics["shortest_path_unweighted"] = nx.shortest_path_length(G, source=first_node, target=last_node)
        metrics["shortest_path_weighted"] = nx.shortest_path_length(G, source=first_node, target=last_node, weight='weight')

    # Number of triangles (3-cliques)
    metrics["num_triangles"] = sum(nx.triangles(G).values()) // 3  # Each triangle counted 3 times

    # Number of tetrahedra (4-cliques)
    metrics["num_tetrahedra"] = avg_tetr_cc(G)  # Ensure avg_tetr_cc is defined

    # Louvain Modularity (Weighted)
    comms = nx.community.louvain_communities(G, weight='weight')
    metrics["modularity_louvain"] = nx.community.modularity(G, comms, weight='weight')

    # Clustering Coefficient
    metrics["clustering_coefficient"] = nx.average_clustering(G, weight='weight')

    # Degree (Max & Mean)
    degrees = dict(G.degree())
    metrics["max_degree"] = max(degrees.values())
    metrics["mean_degree"] = np.mean(list(degrees.values()))

    # Betweenness Centrality (Max & Mean)
    betweenness = nx.betweenness_centrality(G, weight='weight')
    metrics["max_betweenness"] = max(betweenness.values())
    metrics["mean_betweenness"] = np.mean(list(betweenness.values()))

    # Strength (Weighted Degree) (Max & Mean)
    strength = {node: sum(G[node][nbr].get('weight', 1) for nbr in G[node]) for node in G.nodes()}
    metrics["max_strength"] = max(strength.values())
    metrics["mean_strength"] = np.mean(list(strength.values()))

    # Compute Laplacian Eigenvalues
    L = nx.laplacian_matrix(G).toarray()  # Convert sparse matrix to dense NumPy array
    eigenvalues = eigvalsh(L)  # Compute eigenvalues

    if len(eigenvalues) > 1:  # Ensure there are at least two eigenvalues
        metrics["fiedler_value"] = eigenvalues[1]  # Second smallest eigenvalue (λ₂)
        metrics["largest_laplacian_eigenvalue"] = eigenvalues[-1]  # Largest eigenvalue (λ_max)

    return metrics
import hypernetx as hnx
import numpy as np
import networkx as nx
from scipy.linalg import eigvalsh

def compute_hypergraph_metrics(H):
    """
    Computes various hypergraph metrics for a given HyperNetX hypergraph H.

    Metrics:
    - Number of Nodes
    - Number of Hyperedges
    - Max & Mean Hyperedge Size
    - Max & Mean Node Degree
    - Max & Mean Betweenness Centrality
    - Fiedler Value (Second Smallest Eigenvalue of Hypergraph Laplacian)
    - Largest Laplacian Eigenvalue

    Parameters:
    - H (hnx.Hypergraph): A HyperNetX hypergraph.

    Returns:
    - Dictionary with computed hypergraph metrics.
    """
    metrics = {
        "num_hyper_nodes": np.nan,
        "num_hyperedges": np.nan,
        "max_hyperedge_size": np.nan,
        "mean_hyperedge_size": np.nan,
        "hyper_max_node_degree": np.nan,
        "hyper_mean_node_degree": np.nan,
        "hyper_max_betweenness": np.nan,
        "hyper_mean_betweenness": np.nan,
        "hyper_fiedler_value": np.nan,
        "hyper_largest_laplacian_eigenvalue": np.nan
    }

    if not H or len(H.nodes) == 0:
        return metrics

    # Number of nodes and hyperedges
    metrics["num_hyper_nodes"] = len(H.nodes)
    metrics["num_hyperedges"] = len(H.edges)

    # Hyperedge size statistics (how many nodes in each hyperedge)
    hyperedge_sizes = [len(H.edges[he]) for he in H.edges]
    if hyperedge_sizes:
        metrics["max_hyperedge_size"] = max(hyperedge_sizes)
        metrics["mean_hyperedge_size"] = np.mean(hyperedge_sizes)

    # Node degree statistics (how many hyperedges each node belongs to)
    node_degrees = {node: len(H.nodes[node]) for node in H.nodes}
    if node_degrees:
        metrics["hyper_max_node_degree"] = max(node_degrees.values())
        metrics["hyper_mean_node_degree"] = np.mean(list(node_degrees.values()))

    # Convert hypergraph to bipartite graph for centrality analysis
    B = H.bipartite()  # Creates a bipartite graph (nodes + hyperedges)

    # Betweenness centrality on the bipartite representation
    if len(B.nodes) > 1:  # Ensure we have enough nodes
        betweenness = nx.betweenness_centrality(B, weight='weight')
        metrics["hyper_max_betweenness"] = max(betweenness.values())
        metrics["hyper_mean_betweenness"] = np.mean(list(betweenness.values()))

        # Compute Laplacian eigenvalues
        L = nx.laplacian_matrix(B).toarray()  # Get Laplacian matrix
        eigenvalues = eigvalsh(L)  # Compute eigenvalues

        if len(eigenvalues) > 1:
            metrics["hyper_fiedler_value"] = eigenvalues[1]  # Second smallest eigenvalue (λ₂)
            metrics["hyper_largest_laplacian_eigenvalue"] = eigenvalues[-1]  # Largest eigenvalue (λ_max)

    return metrics

def compute_distribution_stats(births, deaths, persistences):
    """
    Given arrays/lists of births, deaths, and persistences, compute summary stats.
    Returns a dict of named metrics.
    """
    births = np.array(births, dtype=float)
    deaths = np.array(deaths, dtype=float)
    pers   = np.array(persistences, dtype=float)
    
    if len(pers) == 0:
        return dict.fromkeys([
            'birth_rate','death_rate','mean_persistence','max_persistence',
            'std_persistence','skewness','kurtosis','entropy'
        ], np.nan)
    
    birth_rate = births.mean()
    death_rate = deaths.mean()
    mean_persistence = pers.mean()
    max_persistence  = pers.max()
    std_persistence  = pers.std(ddof=1)
    skewness = stats.skew(pers, bias=False)
    kurt = stats.kurtosis(pers, bias=False)
    number=len(pers)
    # If you truly want entropy of the raw "pers" values (not a histogram):
    # be aware that stats.entropy(pers) is not standard (it’s for discrete pmf).
    # Typically you'd do a histogram-based approach, but for demonstration:
    #   ent = stats.entropy(pers)
    # Or, a histogram-based approach:
    #   hist, _ = np.histogram(pers, bins='auto', density=True)
    #   ent = stats.entropy(hist) if np.any(hist > 0) else 0.0
    
    ent = stats.entropy(pers)  # Just following your snippet, though it's unusual
    
    return {
        'birth_rate': birth_rate,
        'death_rate': death_rate,
        'mean_persistence': mean_persistence,
        'max_persistence': max_persistence,
        'std_persistence': std_persistence,
        'skewness': skewness,
        'kurtosis': kurt,
        'entropy': ent,
        'number':number
    }


from mpl_toolkits.mplot3d import Axes3D  # needed for 3D plotting
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

%matplotlib inline


def visualize_rips_simplicial_complex(embed, dataset_name, entry, max_edge_length=3.0):
    """
    1) Builds a Rips complex (via GUDHI) from a set of high-dimensional points.
    2) Extracts simplices (up to dimension 2) from the simplex tree.
       - Edges (1-simplices) and triangles (2-simplices).
    3) Uses PCA to reduce the points to 3D.
    4) Plots a 3D visualization:
       - Nodes are shown as a scatter plot.
       - Edges are drawn as lines.
       - Triangles are drawn as filled polygons (using Poly3DCollection).
    
    Parameters:
    -----------
    embed : np.ndarray of shape (N, D)
        The high-dimensional point cloud.
    max_edge_length : float
        The maximum edge length used in the Rips complex.
    """
    # 1) Build the Rips complex and create the simplex tree
    rips_complex = gd.RipsComplex(points=embed, max_edge_length=max_edge_length)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)
    
    # 2) Extract simplices:
    edges = []
    triangles = []
    
    # get_skeleton(2) returns all simplices up to dimension 2
    for simplex, fvalue in simplex_tree.get_skeleton(2):
        if len(simplex) == 2:
            # 1-simplices: edges
            edges.append(simplex)
        elif len(simplex) == 3:
            # 2-simplices: triangles
            triangles.append(simplex)
    
    # 3) Use PCA to reduce the point cloud to 3D
    pca = PCA(n_components=3)
    coords_3d = pca.fit_transform(embed)  # shape (N, 3)
    n_points = coords_3d.shape[0]
    
    # Prepare colormap for nodes (using 'magma_r')
    norm = plt.Normalize(vmin=0, vmax=n_points - 1)
    cmap = plt.get_cmap('plasma_r')
    node_colors = cmap(norm(np.arange(n_points)))
    
    # 4) Create the 3D plot
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot nodes
    sc = ax.scatter(coords_3d[:, 0], coords_3d[:, 1], coords_3d[:, 2],
                    c=node_colors, s=30, alpha=0.9)
    
    # Plot edges as lines
    for edge in edges:
        i, j = edge
        x_vals = [coords_3d[i, 0], coords_3d[j, 0]]
        y_vals = [coords_3d[i, 1], coords_3d[j, 1]]
        z_vals = [coords_3d[i, 2], coords_3d[j, 2]]
        # Optionally, color edge based on one endpoint's index or the average.
        avg_idx = int(np.mean(edge))
        edge_color = cmap(norm(avg_idx))
        ax.plot(x_vals, y_vals, z_vals, color=edge_color, alpha=0.8, linewidth=1.5)
    
    # Plot triangles as filled faces
    face_polys = []
    face_colors = []
    for tri in triangles:
        # Get the 3 vertices for this triangle
        pts = [coords_3d[idx] for idx in tri]
        face_polys.append(pts)
        # Color can be computed from the average index of the triangle's vertices
        avg_idx = int(np.mean(tri))
        face_colors.append(cmap(norm(avg_idx)))
    
    # Create a Poly3DCollection for the triangles with a set transparency (alpha)
    poly_collection = Poly3DCollection(face_polys, alpha=0.3, edgecolor='k')
    poly_collection.set_facecolor(face_colors)
    ax.add_collection3d(poly_collection)
    
    # Set title and labels
    ax.set_title(f"", pad=20)
    ax.set_xlabel("PCA 1")
    ax.set_ylabel("PCA 2")
    ax.set_zlabel("PCA 3")
    
    # Add colorbar for node indices
    sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, pad=0.1)
    cbar.set_label("Node Index")
        # Define three different viewing angles
    angles = [(15, 180), (30, 90), (45, 0)]  # (elevation, azimuth) in degrees
    dir_fig_save=working_dir+f'rips_skeletons/{dataset_name}_{entry}/'
    os.makedirs(dir_fig_save, exist_ok=True)

    # Save figures from different angles
    for i, (elev, azim) in enumerate(angles):
        ax.view_init(elev=elev, azim=azim)  # Set camera angle
        filename = dir_fig_save+f"{window}_{embedding_step}_{i}.png"
        plt.savefig(filename, dpi=300, bbox_inches='tight')  # Save figure
       # print(f"Saved: {filename}")

    #plt.show()



In [114]:

window=128
step=int(window*0.125)
topics=500
nonzeroes=5
embedding_step=int(window*0.5)
#infile = open(f'/home/ll16598/Documents/POSTDOC/Context-DATM/sentenceBERT_context_models_{window}_{embedding_step}/{topics}comp{nonzeroes}nonzeros_dictionary_','rb')
#dictionary=pickle.load(infile)
#infile.close()

save_thresh=244
threshold=save_thresh
# infile = open(f'/home/ll16598/Documents/POSTDOC/Context-DATM/sentenceBERT_cluster_dicts_{window}_{embedding_step}/cluster_dictionary_{save_thresh}','rb')
# cluster_dictionary=pickle.load(infile)
# infile.close()

dir_array='/home/ll16598/Documents/POSTDOC/vector_assigned_dfs'
dir_atom_dfs='/home/ll16598/Documents/POSTDOC/Context-DATM/atom_assigned_dfs'

#df_drug=pd.read_csv(f'.{}/df_monolog_{threshold}.csv')

df_monologs=pd.read_csv(f'{dir_atom_dfs}/df_monolog_{threshold}.csv')

layers='last'
with open(f'{dir_array}/{window}_{step}_SER_monologs_sentence_embeddings_arrays.pkl', 'rb') as f:
    df_monologs['sentence_embeddings'] = pickle.load(f)
    
df_SER=pd.read_csv(f'{dir_atom_dfs}/df_SER_{threshold}.csv')
with open(f'{dir_array}/{window}_{step}_SER1_sentence_embeddings_arrays.pkl', 'rb') as f:
    df_SER['sentence_embeddings'] = pickle.load(f)
df_SER2=pd.read_csv(f'{dir_atom_dfs}/df_SER2_{threshold}.csv')
with open(f'{dir_array}/{window}_{step}_SER_IPSP_sentence_embeddings_arrays.pkl', 'rb') as f:
    SER2_array= pickle.load(f)
    df_SER2['sentence_embeddings'] =SER2_array

In [ ]:
dfs=[df_monologs, df_SER, df_SER2]
df_names=['monolog', 'SER', 'SER2']
for df_no, df_monolog in enumerate(dfs):
    df_name=df_names[df_no]
    df_monolog['token_embeddings']=None
    embeddings='sentence_embeddings'
    drugs=sorted(list(set(df_monolog['Drug'])))
    Participants=sorted(list(set(df_monolog['Participant'])))
    working_dir='/home/ll16598/Documents/POSTDOC/Context-DATM/'
    data_save_dir=working_dir+'TDA_output/'
    os.makedirs(data_save_dir, exist_ok=True)

    df_monolog=get_rips_time_centroid(df_monolog,embeddings=embeddings)
    for D in [0,1]:
        df_monolog=get_rips_time(df_monolog,embeddings=embeddings, D=D)

        df_exploded = df_monolog.explode([f"scales_dim{D}", f'alive_dim{D}'])
        df_exploded[f"scales_dim{D}"] = pd.to_numeric(df_exploded[f"scales_dim{D}"])
        df_exploded[f'alive_dim{D}'] = pd.to_numeric(df_exploded[f'alive_dim{D}'])
        grouped = df_exploded.groupby(["Drug", f"scales_dim{D}"], as_index=False).agg(
            alive_mean=(f'alive_dim{D}', "mean"),
            alive_se=(f'alive_dim{D}', sem)  # standard error
        )
        df_exploded.to_csv(data_save_dir+f'{df_name}_{D}_simplices_over_time.csv')


    df_centroid_exploded = df_monolog.explode(["centroid_scales_dim0", 'centroid_alive_dim0'])
    df_centroid_exploded[f"centroid_scales_dim0"] = pd.to_numeric(df_centroid_exploded[f"centroid_scales_dim0"])
    df_centroid_exploded[f'centroid_alive_dim0'] = pd.to_numeric(df_centroid_exploded[f'centroid_alive_dim0'])
    grouped_centroid = df_centroid_exploded.groupby(["Drug", f"centroid_scales_dim0"], as_index=False).agg(
        alive_mean=(f'centroid_alive_dim0', "mean"),
        alive_se=(f'centroid_scales_dim0', sem)  # standard error
    )
    df_centroid_exploded.to_csv(data_save_dir+f'{df_name}_centroid_simplices_over_time.csv')


    df_monolog=get_rips_time(df_monolog,embeddings=embeddings, D=0)

    df_with_graph=get_rips_complex_G(df_monolog)
    # Apply functions to each graph in the dataframe
    df_with_graph['sorted_nodes'], df_with_graph['fvalues_between_nodes'] = zip(
        *df_with_graph['graph'].apply(get_node_join_fvalues)
    )

    # Compute Pearson & Spearman correlations and store them in new columns
    df_with_graph[['pearson_corr', 'spearman_corr']] = df_with_graph.apply(
        lambda row: compute_correlations(row['sorted_nodes'], row['fvalues_between_nodes']), axis=1, result_type='expand'
    )

    df_with_graph['euler'] = df_with_graph['simplex_tree'].apply(lambda st: compute_euler_characteristic(st, max_dim=3))


    # Apply the function to each graph in df_with_graph
    graph_metrics = df_with_graph['graph'].apply(compute_graph_metrics)

    # Convert dictionary results into separate columns
    graph_metrics_df = pd.DataFrame(graph_metrics.tolist())

    # Merge with original dataframe
    df_with_graph = pd.concat([df_with_graph, graph_metrics_df], axis=1)

    hyper_graph_metrics = df_with_graph['hypergraph'].apply(compute_hypergraph_metrics)

    # Convert dictionary results into separate columns
    hyper_graph_metrics_df = pd.DataFrame(hyper_graph_metrics.tolist())

    # Merge with original dataframe
    df_with_graph = pd.concat([df_with_graph, hyper_graph_metrics_df], axis=1)


    # List out the homology dimensions you want
    dimensions = [0, 1, 2]

    # We'll accumulate new rows in a list of dicts
    new_rows = []

    for idx, row in df_with_graph.iterrows():
        # You might have an "embedding" column with the data. 
        # Or, if you have multiple columns, you might do something like:
        # embed = row[embeddings].values (depending on your structure).
        embed = row[embeddings]  # Adjust as needed

        # Build the Rips Complex for *this row only*
        rips_complex = gd.RipsComplex(points=embed, max_edge_length=5.0)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)
        persistence = simplex_tree.persistence()

        # We’ll store births, deaths, pers LENGTHS in a dict keyed by dimension
        dim_dict = {
            dim: {'births': [], 'deaths': [], 'pers': []}
            for dim in dimensions
        }

        # Collect intervals by dimension
        for dim, (b, d) in persistence:
            if d == float('inf'):
                continue
            if dim in dimensions:
                dim_dict[dim]['births'].append(b)
                dim_dict[dim]['deaths'].append(d)
                dim_dict[dim]['pers'].append(d - b)

        # Now compute summary stats for each dimension.
        # We can store them in one of two ways:
        # Option A: Put *all dimensions* stats in separate columns of a single row
        # Option B: Create separate rows for each dimension (then you keep dimension as a column)
        #
        # Let's do Option A: store dimension 0, 1, 2 in separate columns (like birth_rate_dim0, etc.).

        row_dict = row.to_dict()  # Start with original row's columns

        for dim in dimensions:
            bdp = dim_dict[dim]
            stats_dict = compute_distribution_stats(bdp['births'], bdp['deaths'], bdp['pers'])
            # prefix each stat key with dim
            for stat_key, stat_val in stats_dict.items():
                row_dict[f"{stat_key}_dim{dim}"] = stat_val

        # Add row_dict to new_rows
        new_rows.append(row_dict)

    # Create a new DataFrame
    df_with_tda = pd.DataFrame(new_rows)
    df_with_tda.to_csv(data_save_dir, f'{df_name}_TDA_results')
    print(f'completed! {df_name}')
    for entry in range(0, len(df_with_tda['sentence_embeddings'])):
        if not df_with_tda['graph'][entry]:
            continue
        else:
            visualize_rips_simplicial_complex(df_with_tda['sentence_embeddings'][entry], df_name, entry, max_edge_length=df_with_tda['rt'][entry])

